In [ ]:
# usual imports
import os
import numpy as np
import matplotlib.pyplot as plt
from rail.core.utils import RAILDIR
#from rail.estimation.algos.bpz_version.utils import RAIL_BPZ_DIR
from rail.pipelines.examples.survey_nonuniformity.survey_nonuniformity import SurveyNonuniformDegraderPipeline
from rail.core import common_params
from rail.pipelines.utils.name_factory import NameFactory, DataType, CatalogType, ModelType, PdfType
import qp
import ceci


In [ ]:
#change to your rail location
RAIL_BPZ_DIR = "/path/to/your/rail/src/"

### Define the bands, magnitude limits, and put them into a dictionary

In [5]:
bands = 'grizy'
maglims = [27.79, 29.04, 29.06, 28.62, 27.98, 27.05]
maglim_dict={}
for band,limx in zip(bands, maglims):
    maglim_dict[f"mag_{band}_lsst"] = limx

### Setting common parameters to all the stages in the estimation


In [ ]:
common_params.set_param_defaults(
    bands=[f'mag_{band}_lsst' for band in bands], # bands
    err_bands=[f'mag_err_{band}_lsst' for band in bands], # uncertainty bands
    nondetect_val=np.nan,
    #ref_band='HSCi_cmodel_dered',
    redshift_col='redshift',
    mag_limits=maglim_dict,
    prior_band="i",
    #zmax=6.0, # set zmax to 6 for HSC
)

### Set up the pipeline

In [ ]:
pipe = SurveyNonuniformDegraderPipeline()

In [ ]:
namer = NameFactory()

### Define parameters for each stage

In [ ]:
# dictionary used by ObsCondition
# first define a set of input map directories:

base_path = "/pscratch/sd/q/qhang/rubin_baseline_v2/MAF-1year/"

# nside of these maps:
nside=128

# seeing maps:
seeing_u = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_u_and_nightlt365_HEAL.fits"
seeing_g = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_g_and_nightlt365_HEAL.fits"
seeing_r = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_r_and_nightlt365_HEAL.fits"
seeing_i = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_i_and_nightlt365_HEAL.fits"
seeing_z = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_z_and_nightlt365_HEAL.fits"
seeing_y = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_y_and_nightlt365_HEAL.fits"

# coadd depth maps:
coaddm5_u = base_path + "baseline_v2_0_10yrs_CoaddM5_u_and_nightlt365_HEAL.fits"
coaddm5_g = base_path + "baseline_v2_0_10yrs_CoaddM5_g_and_nightlt365_HEAL.fits"
coaddm5_r = base_path + "baseline_v2_0_10yrs_CoaddM5_r_and_nightlt365_HEAL.fits"
coaddm5_i = base_path + "baseline_v2_0_10yrs_CoaddM5_i_and_nightlt365_HEAL.fits"
coaddm5_z = base_path + "baseline_v2_0_10yrs_CoaddM5_z_and_nightlt365_HEAL.fits"
coaddm5_y = base_path + "baseline_v2_0_10yrs_CoaddM5_y_and_nightlt365_HEAL.fits"

# here we will set the observing year and number of visits per year to 1, because we are supplying coadd depth

# mask:
maskdir = base_path + "../wfd_footprint_nvisitcut_500_nside_128.fits"

# weight: for now we supply uniform weight

# choose the systematic map to examine, here we choose the combined depth:
sys_to_check = base_path + "baseline_v2_0_10yrs_CoaddM5_i_and_nightlt365_HEAL.fits"
sys = "CoaddM5"

# directory to save all the data:
savedir = "/pscratch/sd/q/qhang/PZflow-samples/DC2-test/"

map_dict={
        "theta": {
            "u": seeing_u,
            "g": seeing_g,
            "r": seeing_r,
            "i": seeing_i,
            "z": seeing_z,
            "y": seeing_y,
        },
        "m5": {
            "u": coaddm5_u,
            "g": coaddm5_g,
            "r": coaddm5_r,
            "i": coaddm5_i,
            "z": coaddm5_z,
            "y": coaddm5_y,
        },
        "nYrObs": 1.,
        "nVisYr": {
            "u": 1.,
            "g": 1.,
            "r": 1.,
            "i": 1.,
            "z": 1.,
            "y": 1., 
        },
        "sigLim": 10,
        "ndFlag": np.nan,
        "extendedType": "auto",
        "majorCol": "major",
        "minorCol": "minor",
        "decorrelate": True,
        "highSNR": False,
        "renameDict": band_dict,  
    }

In [ ]:
pipe.flow_engine_train.config.update(n_samples=20)
pipe.obs_condition.config.update(map_dict=map_dict, nside=nside, mask = maskdir, weight = "",)
#pipe.estimate_bpz.config.update(spec_groupname = "", redshift_colname = 'specz_redshift')

In [ ]:
pipe_info = pipe.initialize(input_dict, dict(output_dir='.', log_dir='.', resume=True), None)

### Save the pipeline into a yml file

In [ ]:
pipe.save('tmp_survey_nonuniformity.yml')

### run pipeline

In [ ]:
import ceci
pr = ceci.Pipeline.read('tmp_survey_nonuniformity.yml')

In [ ]:
pr.run()

# Redshift distribution in each tomographic bins at different depth:

In [ ]:
# Various rail modules
import rail.stages
rail.stages.import_and_attach_all()
from rail.stages import *

In [ ]:
### find where is Dereddener!
Dereddener.__module__